In [1]:
!rm -rf /home/gdholla1/projects/post_mortem/workflow_folders/t2star/

In [13]:
def fit_t2(t2w, tes, output=None):
    import numpy as np
    import nibabel as nb
    import os
    
    if not output:
        output = os.path.abspath('t2.nii.gz')
    
    tes = np.array(tes)    
    x = np.concatenate((np.ones_like(tes[..., np.newaxis]), -tes[..., np.newaxis]), 1)
    
    flash_images = [nb.load(fn) for fn in t2w]
    flashes = [flash.get_data() for flash in flash_images]
    
    data = np.log(flashes)
    data[data < 0 ] = 1e-6
    
    beta = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(data.reshape(data.shape[0], np.prod(data.shape[1:]))).reshape((2,) + data.shape[1:])

    s0 = np.exp(beta[0])
    t2 = 1./beta[1]
    
    nb.save(nb.Nifti1Image(t2, flash_images[0].affine), output)
    
    pd = os.path.abspath('pd.nii.gz')
    nb.save(nb.Nifti1Image(s0, flash_images[0].affine), pd)
    
    return output, pd

In [14]:
import nipype.pipeline.engine as pe

In [15]:
import glob

In [16]:
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces import utility as util

In [30]:
workflow = pe.Workflow(name='t2star')
workflow.base_dir = '/home/gdholla1/projects/post_mortem/workflow_folders'

subject_ids = [13095, 14037, 14051, 14069, 15033, 15035, 15055]
subject_ids += [12062, 12082, 12104]

tes = [4.67, 11.77, 18.87, 25.97, 33.07, 40.17, 47.27, 54.37]
templates = {'t2w':'/home/gdholla1/projects/post_mortem/data/raw/T2s_map/{subject_id}/{subject_id}_S*_FLASH_3D_as_gre_0p20_cor_8echos_T2sMap*_Te{te}.nii'}

selector = pe.MapNode(nio.SelectFiles(templates), iterfield=['te'], name='selector')
selector.inputs.te = tes
selector.iterables = [('subject_id', subject_ids)]

t2_fitter = pe.Node(util.Function(function=fit_t2, 
                                  input_names=['t2w', 'tes'],
#                                   ignore_exception=True,
                                  output_names=['t2', 'pd']),
                   name='t2_fitter')



t2_fitter.inputs.tes = tes

workflow.connect(selector, 't2w', t2_fitter, 't2w')

ds = pe.Node(nio.DataSink(base_directory='/home/gdholla1/projects/post_mortem/data/derivatives'), name='datasink')
workflow.connect(t2_fitter, 't2', ds, 't2maps')
workflow.connect(t2_fitter, 'pd', ds, 'proton_density')

workflow.run()


<string>:15: RuntimeWarning: divide by zero encountered in log


In [31]:
import nibabel as nb

In [32]:
import shutil

for subject_id in subject_ids:
    
    old_fn = '/home/gdholla1/projects/post_mortem/data/derivatives/t2maps/_subject_id_{subject_id}/t2.nii.gz'.format(**locals())
    new_fn = '/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/Control/T2s_map/{subject_id}/{subject_id}_t2map.nii.gz'.format(**locals())
    shutil.copy(old_fn, new_fn)
    
    
    old_fn = '/home/gdholla1/projects/post_mortem/data/derivatives/proton_density/_subject_id_{subject_id}/pd.nii.gz'.format(**locals())
    new_fn = '/home/public/HISTO_TO_MRI_1/MRI_DATA/MRI/Control/T2s_map/{subject_id}/{subject_id}_pd.nii.gz'.format(**locals())
    shutil.copy(old_fn, new_fn)    